In [ ]:
#data analysis libraries
import pandas as pd
import numpy as np

#visualization libraries

import matplotlib.pyplot as plt 
import seaborn as sns 
import plotty.express as px 
from scipy.stats import pearsonr

# Set the default figure size for Seaborn plots globally
sns.set(rc={'figure.figsize':(10,6)})
%matplotlib inline 